In [ ]:
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd

In [ ]:
import psycopg2


In [ ]:
conn_string = 'host={0} user={1} dbname={2} password={3} sslmode={4}'.format(host, user, dbname, password, sslmode)

In [ ]:
headers = ({'User-Agent' : 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:102.0) Gecko/20100101 Firefox/102.0'})
url = 'https://df.olx.com.br/imoveis/aluguel'
response = rq.get(url,  headers=headers)
content = response.content

site = bs(content, 'html.parser')

In [ ]:
l = site.find('div', class_='sc-EHOje gpDieb')

paginas = []


for item in l.find_all('a'):
  paginas.append(item['href'])
paginas

In [ ]:
titulos = []
links = []
valores = []


itens = site.find('ul', id='ad-list')
for pagina in paginas:
  for item in itens.find_all('a'):
    links.append(item['href'])
  else:
    links.append('') 

In [ ]:
itens = site.find('ul', id='ad-list')
for pagina in paginas:
  for item in itens.find_all('a'):
    titulos.append(item['title'])
  else:
    titulos.append('')

In [ ]:
itens = site.find('ul', id='ad-list')
for pagina in paginas:
  for item in itens.find_all('a'):
    titulos.append(item['title'])
  else:
    titulos.append('')

In [ ]:
localizacao = []


itens = site.find('ul', id='ad-list')
for pagina in paginas:

  for item in itens.find_all('span', class_='sc-1c3ysll-1 cLQXSQ sc-ifAKCX lgjPoE'):
    localizacao.append(item['title'])
  else:
    localizacao.append('') 

In [ ]:
data = []

itens = site.find('ul', id='ad-list')
for pagina in paginas:


  
    for item in itens.find_all('span', class_='sc-11h4wdr-0 javKJU sc-ifAKCX lgjPoE'):
      data.append(item['aria-label'])
    else:
      data.append('')  

In [ ]:
metros = []
quartos = []
cond = []

itens = site.find('ul', id='ad-list')
for pagina in paginas:
  for item in itens.find_all('span', class_='sc-1ftm7qz-0 doofcG sc-ifAKCX lgjPoE'):
      if 'm²' in str(item.text):
        metros.append(item.text)
      elif 'quartos' in str(item.text):
        quartos.append(item.text)
      elif 'Condomínio' in str(item.text):
        cond.append(item.text)
          

In [ ]:
df = pd.DataFrame(list(zip(titulos, links, valores, localizacao, data, metros, quartos, cond,)),
                  columns=['titulo', 'link', 'aluguel', 'localização', 'data de postagem', 'metragem', 'quartos', 'condominio' ])

In [ ]:
df['aluguel'] = df['aluguel'].str.replace(r'\D', '')
df['condominio'] = df['condominio'].str.replace(r'\D', '')
df['metragem'] = df['metragem'].str.replace(r'\D', '')
df['quartos'] = df['quartos'].str.replace(r'\D', '')


In [ ]:
df['condominio'] = pd.to_numeric(df['condominio'])
df['metragem'] = pd.to_numeric(df['metragem'])
df['aluguel'] = pd.to_numeric(df['aluguel'])
df['quartos'] = pd.to_numeric(df['quartos'])

In [ ]:
df['valor total'] = df['aluguel'] + df['condominio']

In [ ]:
#adicionando os dados ao postgres
import sqlalchemy
from sqlalchemy import create_engine

In [ ]:
conn_string = 'jdbc:postgresql://locahost5432/olx'

In [ ]:
PGHOST = 'localhost'
PGDATABASE = 'olx'
PGUSER = 'postgres'
PGPASSWORD = 'senha'

In [ ]:
engine = sqlalchemy.create_engine(f'postgresql://{PGUSER}:{PGPASSWORD}@{PGHOST}:5432/{PGDATABASE}')

In [ ]:
df.to_sql('df', engine, if_exists='append', index=False)